In [1]:
!pip install  ultralytics==8.0.43

In [25]:
!pip install matplotlib

In [27]:
import Augmentor
import os
import shutil
from shutil import SameFileError
def augment_data(pth, num_samples, augment_factor=2):
    # Iterate through sub-directories
    for subdir in os.listdir(pth):
        subdir_path = os.path.join(pth, subdir)
        if os.path.isdir(subdir_path):
            output_dir = subdir_path  # Specify the output directory
            os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

            # Create Augmentor pipeline for each sub-directory
            pipeline = Augmentor.Pipeline(subdir_path, output_directory=output_dir)

            # Define augmentations
            pipeline.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
            pipeline.flip_left_right(probability=0.7)
            pipeline.zoom_random(probability=0.3, percentage_area=0.8)
            pipeline.flip_top_bottom(probability=0.5)
            print(num_samples)
            # Number of samples per original image
            pipeline.sample(num_samples)
            
            # Further augment the data by applying the pipeline multiple times
            for i in range(augment_factor - 1):
                pipeline.process()

            # Copy original images to the output directory
            original_images = os.listdir(subdir_path)
            for img in original_images:
                try:
                    shutil.copy(os.path.join(subdir_path, img), output_dir)
                except SameFileError:
                    continue

num_samples_per_image = 4 # Number of samples to generate for each original image
augment_factor = 3
path='C:/Users/TOPINFORMATIQUE/Desktop/test3/valid'
print(path)
# Make sure to define the 'path' variable with the correct path before calling the function
augment_data(path, num_samples=num_samples_per_image, augment_factor=augment_factor)

C:/Users/TOPINFORMATIQUE/Desktop/test3/valid
Initialised with 42 image(s) found.
Output directory set to C:/Users/TOPINFORMATIQUE/Desktop/test3/valid\fallen.4


Processing <PIL.Image.Image image mode=RGB size=640x640 at 0x1D957064070>: 100%|███| 4/4 [00:00<00:00, 31.00 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=640x640 at 0x1D957057700>: 100%|█| 42/42 [00:00<00:00, 95.22 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=640x640 at 0x1D957052610>: 100%|█| 42/42 [00:00<00:00, 95.87 Samples/s]


Initialised with 78 image(s) found.
Output directory set to C:/Users/TOPINFORMATIQUE/Desktop/test3/valid\standing.4


Processing <PIL.Image.Image image mode=RGB size=640x640 at 0x1D957022C10>: 100%|███| 4/4 [00:00<00:00, 27.58 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=640x640 at 0x1D95706C5E0>: 100%|█| 78/78 [00:00<00:00, 86.55 Samples/s]
Processing <PIL.Image.Image image mode=RGB size=640x640 at 0x1D95706A820>: 100%|█| 78/78 [00:00<00:00, 93.84 Samples/s]


In [29]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths to your dataset
train_dir = 'C:\\Users\\TOPINFORMATIQUE\\Desktop\\test3\\train'
validation_dir = 'C:\\Users\\TOPINFORMATIQUE\\Desktop\\test3\\valid'

# Image data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Normalize images
    rotation_range=20,           # Data augmentation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to fit model input
    batch_size=64,
    class_mode='binary'      # Binary classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)


Found 1700 images belonging to 2 classes.
Found 368 images belonging to 2 classes.


In [30]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator
)

# Save the trained model
model.save('resnet50_finetuned.h5')

Epoch 1/50


C:\Users\TOPINFORMATIQUE\anaconda3\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 123s 4s/step - accuracy: 0.5059 - loss: 5.6507 - val_accuracy: 0.6467 - val_loss: 0.6012
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.6217 - loss: 0.6460 - val_accuracy: 0.3696 - val_loss: 1.1353
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step - accuracy: 0.5995 - loss: 0.6746 - val_accuracy: 0.6957 - val_loss: 0.5793
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step - accuracy: 0.7531 - loss: 0.5520 - val_accuracy: 0.6522 - val_loss: 0.5852
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 101s 3s/step - accuracy: 0.7569 - loss: 0.5290 - val_accuracy: 0.7065 - val_loss: 0.5416
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 100s 3s/step - accuracy: 0.7337 - loss: 0.5211 - val_accuracy: 0.7120 - val_loss: 0.5191
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 100s 3s/step - accuracy: 0.7652 - loss: 0.4884 - val_accuracy: 0.7310 - val_loss: 0.4896
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 102s 3s/step - accuracy: 0.8108 - loss: 0.4516 - val_accuracy: 0.8098 - val_loss: 0.

In [31]:
# Load the model
model = tf.keras.models.load_model('resnet50_finetuned.h5')

# Classify a new image
def classify_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    return 'standing' if prediction[0][0] > 0.5 else 'fallen'

In [36]:
result = classify_image('C:/Users/TOPINFORMATIQUE/Desktop/fall-labeled/test/images/image_246_jpg.rf.87fddb38ee3a41713bea90274a21868c.jpg')
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
standing
